#ResNext + LSTM 모델을 활용

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

import glob
import torch
import torchvision
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=26bb28769174d57fd3347a340996d9eb382cd6e9387990efde1cb2cdb40372f8
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import face_recognition

# Function to check if an image file is corrupted or not -> 필요없어졌는 지 check
def is_corrupted_image(image_path):
    try:
        _ = cv2.imread(image_path)
        return False
    except Exception as e:
        print(f"Corrupted image: {image_path}")
        return True

# Function to validate a list of image files
def validate_images(image_paths, train_transforms):
    validated_images = []
    for image_path in image_paths:
        if not is_corrupted_image(image_path):
            try:
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB, openCv가 적용하는 image 특성은 BGR이다.
                pil_image = Image.fromarray(image) #OpenCv가 읽는 image를 array형태 numerical한 형태 객체인 파이썬 라이브러리를 이용한 PIL 객체로 변환 
                pil_image = train_transforms(pil_image) #앞서 만들어둔 image transformation을 적용.
                validated_images.append(pil_image)
            except Exception as e:
                print(f"Error processing image: {image_path}")
                continue
    return validated_images

# Constants and configurations
#im_size = 112
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Define transformations
train_transforms = transforms.Compose([
    #transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std) #표준화 이용. 이미지 정규화 하는 부분에서는 Normalization (0-1) 사이의 값, 혹은 Min-Max (-1,1) 사이의 값들을 이룬다.
])

# Paths to image directories, #list형태 사용
train_image_dirs = [ 
    '/content/drive/MyDrive/datasets/train/real/',
    '/content/drive/MyDrive/datasets/train/fake/'
]

test_image_dirs = [
    '/content/drive/MyDrive/datasets/test/real/',
    '/content/drive/MyDrive/datasets/test/fake/'
]

# Get list of all train image files
train_image_files = []  #image file들을 저장.(훈련 데이터만 저장을 한다)
for dir_path in train_image_dirs:
    train_image_files += glob.glob(os.path.join(dir_path, '*.jpg'))  # Assuming images are JPEG format

random.shuffle(train_image_files)
random.shuffle(train_image_files)

#Get list of all test image files
test_image_files = []  #image file들을 저장.(테스트 데이터만 저장을 한다)
for dir_path in test_image_dirs:
    test_image_files += glob.glob(os.path.join(dir_path, '*.jpg'))  # Assuming images are JPEG format

random.shuffle(test_image_files)
random.shuffle(test_image_files)

print("Total number of images:", len(train_image_files)) #train_image_files 2x10000
print("Total number of images:", len(test_image_files)) #test_image_files은 2x10000 

# Validate images
validated_images = validate_images(train_image_files, train_transforms)

print("Total number of validated images:", len(validated_images))

In [ ]:
# load the video name and labels from csv
class video_dataset(Dataset):
    def __init__(self,video_names,labels,sequence_length = 60,transform = None):
        self.video_names = video_names
        self.labels = labels
        self.transform = transform
        self.count = sequence_length
    def __len__(self):
        return len(self.video_names)
    def __getitem__(self,idx):
        video_path = self.video_names[idx]
        frames = []
        a = int(100/self.count)
        first_frame = np.random.randint(0,a)
        temp_video = video_path.split('/')[-1]
        #print(temp_video)
        label = self.labels.iloc[(labels.loc[labels["file"] == temp_video].index.values[0]),1]
        if(label == 'FAKE'):
          label = 0
        if(label == 'REAL'):
          label = 1
        for i,frame in enumerate(self.frame_extract(video_path)):
          frames.append(self.transform(frame))
          if(len(frames) == self.count):
            break
        frames = torch.stack(frames)
        frames = frames[:self.count]
        #print("length:" , len(frames), "label",label)
        return frames,label
    def frame_extract(self,path):
      vidObj = cv2.VideoCapture(path) 
      success = 1
      while success:
          success, image = vidObj.read()
          if success:
              yield image
#plot the image
def im_plot(tensor):
    image = tensor.cpu().numpy().transpose(1,2,0)
    b,g,r = cv2.split(image)
    image = cv2.merge((r,g,b))
    image = image*[0.22803, 0.22145, 0.216989] +  [0.43216, 0.394666, 0.37645]
    image = image*255.0
    plt.imshow(image.astype(int))
    plt.show()

In [ ]:
#count the number of fake and real videos
def number_of_real_and_fake_videos(data_list):
  header_list = ["original_path","id","label","label_str","path"]
  lab = pd.read_csv('/content/drive/MyDrive/datasets/labels/train.csv',names=header_list)
  real = 0
  fake = 0
  #csv파일을 읽었을 때 real이 몇 개인지, fake가 몇 개인지
  print(lab[lab['label']].unique()) #1,0
  print(lab[lab['label_str']].unique()) #real, fake

  #만약에 lab['label']이 1이면 real +=1, 0이면 fake +=0
  for label in lab['label']:
        if label == 1:
            real += 1
        else:
            fake += 1
            
  return real,fake

In [ ]:
# load the labels and video in data loader
import random
import pandas as pd
from sklearn.model_selection import train_test_split

header_list = ["original_path","id","label","label_str","path"] #우리의 csv파일에 맞게 label list형태를 변환.
labels = pd.read_csv('/content/drive/MyDrive/datasets/labels/train.csv',names=header_list)
#print(labels)
#train image, validation, test image 만드는 코드 (image_file에는 2만장 real, fake가 섞여있는 객체이고 2만장을 8:2로 각각 train : 16000, validation: 4000장)
#test image도 4천장 정도 만들어야 한다.

train_images = train_image_files[:int(0.8*len(train_image_files))] #처음부터 80%까지의 data를 저장
valid_images = train_image_files[int(0.8*len(train_image_files)):] #20% data는 validation image로 저장.
test_images = test_image_files[:int(0.2*len(test_image_files))] #20% 즉 4000장 정도 -> 나중에 train_test_split을 통해서 real과 fake의 비율을 맞추는 개선 활동 필요.

print("train : " , len(train_images))
print("test : " , len(test_images))
# train_images,valid_images = train_test_split(image_files,test_size = 0.2)
# print(train_videos)

print("TRAIN: ", "Real:",number_of_real_and_fake_videos(train_images)[0]," Fake:",number_of_real_and_fake_videos(train_images)[1])
print("TRAIN: ", "Real:",number_of_real_and_fake_videos(valid_images)[0]," Fake:",number_of_real_and_fake_videos(valid_images)[1])
print("TEST: ", "Real:",number_of_real_and_fake_videos(test_images)[0]," Fake:",number_of_real_and_fake_videos(test_images)[1])


im_size = 112
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])

test_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])
train_data = video_dataset(train_videos,labels,sequence_length = 10,transform = train_transforms)
#print(train_data)
val_data = video_dataset(valid_videos,labels,sequence_length = 10,transform = train_transforms)
train_loader = DataLoader(train_data,batch_size = 4,shuffle = True,num_workers = 4)
valid_loader = DataLoader(val_data,batch_size = 4,shuffle = True,num_workers = 4)
image,label = train_data[0]
im_plot(image[0,:,:,:])